In [59]:
import datetime
import time
import traceback

import pandas as pd
import tushare as ts

# 行业分析

# pro = ts.pro_api()
from dateutil.relativedelta import relativedelta


def get_mean_by_time(ts_code, start_date_in, end_date_in):
    """
    根据时间获取平均数
    :param ts_code:代码
    :param start_date_in:开始时间
    :param end_date_in:结束时间
    :return:时间段内容
    """
    df = pro.daily(ts_code=ts_code, start_date=start_date_in, end_date=end_date_in)
    ser_mean = df['close'].groupby(by=df['ts_code']).mean()
    dict_mean = {'ts_code': ser_mean.index, start_date_in: ser_mean.values}
    df_mean = pd.DataFrame(dict_mean)
    df_mean = df_mean.set_index('ts_code')
    return df_mean


tu_token = 'fba0f69c02f5e4399b8dd63ae895ca69b7a55122a545c6f0f2c2bef3'
ts.set_token(tu_token)
pro = ts.pro_api(tu_token)

# 目标
obj_ts_code = "002223.SZ"

# 结束时间
end_date = '20211001'

# 目标行业
pro.stock_basic(exchange='', fields='industry')

# 行业
trade = pro.stock_basic(exchange='', ts_code=obj_ts_code, fields='industry').iloc[0]['industry']

#查询当前所有正常上市交易的股票列表
data = pro.stock_basic(exchange='', fields='ts_code,name,area,industry,market,list_date')
data = data[data['industry'] == trade]
ts_codes = [obj_ts_code]

# 获取同行业数据
for index, row in data.iterrows():
    if not ts_codes.__contains__(row['ts_code']):
        ts_codes.append(row['ts_code'])

# print(data)
print(ts_codes)
data = data.set_index('ts_code', drop=True)
data.to_csv("../%s.csv" % trade)

str_dates = ['20190101']
start_date = datetime.datetime.strptime('20200101', '%Y%m%d')

# 获取时间列表
for i in range(0, 100):
    tmp_date = start_date + relativedelta(month=i % 12 + 1, years=i // 12)
    if tmp_date > datetime.datetime.strptime(end_date, '%Y%m%d'):
        break
    else:
        s_start_date = tmp_date.strftime("%Y%m%d")
        str_dates.append(s_start_date)


def get_data_by_ts_code(ts_code):
    df_res = get_mean_by_time(ts_code, str_dates[0], str_dates[1])
    for j in range(1, len(str_dates) - 1):
        df_tmp = get_mean_by_time(ts_code, str_dates[j], str_dates[j + 1])
        df_res = df_res.join(df_tmp)
    return df_res

print("count = %d" % len(ts_codes) )

df_res = get_data_by_ts_code(ts_codes[0])
for i in range(1, len(ts_codes)):
    print(f'i = {i} : {ts_codes[i]}')
    # 降低频率
    time.sleep(10)
    try :
        df_tmp = get_data_by_ts_code(ts_codes[i])
        df_res = df_res.append(df_tmp)
    except Exception as e:
        traceback.print_exc()
    # print(df_tmp)

data = pro.stock_basic(exchange='', fields='ts_code,name')
data = data.set_index('ts_code', drop=True)
df_res = data.join(df_res, how='inner')
df_res.to_csv(f'../{trade}-整体.csv')
df_res

['002223.SZ', '000150.SZ', '000503.SZ', '000509.SZ', '000516.SZ', '000710.SZ', '002022.SZ', '002044.SZ', '002173.SZ', '002382.SZ', '002432.SZ', '002524.SZ', '002551.SZ', '002614.SZ', '002901.SZ', '002932.SZ', '002950.SZ', '300003.SZ', '300015.SZ', '300030.SZ', '300049.SZ', '300143.SZ', '300146.SZ', '300171.SZ', '300206.SZ', '300238.SZ', '300244.SZ', '300246.SZ', '300247.SZ', '300273.SZ', '300298.SZ', '300314.SZ', '300318.SZ', '300326.SZ', '300347.SZ', '300358.SZ', '300396.SZ', '300404.SZ', '300406.SZ', '300412.SZ', '300439.SZ', '300453.SZ', '300463.SZ', '300482.SZ', '300529.SZ', '300562.SZ', '300595.SZ', '300633.SZ', '300639.SZ', '300642.SZ', '300653.SZ', '300676.SZ', '300677.SZ', '300753.SZ', '300760.SZ', '300791.SZ', '300813.SZ', '300832.SZ', '300869.SZ', '300888.SZ', '300896.SZ', '300981.SZ', '301033.SZ', '301047.SZ', '301060.SZ', '600055.SH', '600381.SH', '600529.SH', '600530.SH', '600532.SH', '600587.SH', '600763.SH', '600767.SH', '600896.SH', '603127.SH', '603301.SH', '603309.SH'

,name,20190101,20200101,20200201,20200301,20200401,20200501,20200601,20200701,20200801,...,20201201,20210101,20210201,20210301,20210401,20210501,20210601,20210701,20210801,20210901
ts_code,,,,,,,,,,,,,,,,,,,,,
002173.SZ,创新医疗,7.064221,5.88125,5.7765,5.771304,5.370952,5.272632,5.395238,6.051304,7.571818,...,6.563478,5.580476,5.73375,6.4575,7.079524,8.113158,7.005,7.06,6.595217,6.426
